Um problema importantíssimo que Varejistas e outros negócios enfrentam hoje em dia é relacionado a fraudes no cartão de crédito. Inclusive eu mesmo já tendo sido vítima deste tipo de problema.

Podemos aplicar Machine Learning para ajudar a detectar fraudes com cartão de crédito, mas há um pequeno problema em que a grande maioria das transações é perfeitamente legítima, o que reduz a sensibilidade de um modelo típico a fraudes.

Como exemplo, considere um algoritmo logístico em execução no conjunto de dados Fraude de Cartão de Crédito publicado no Kaggle. Você pode baixá-lo aqui:

https://www.kaggle.com/mlg-ulb/creditcardfraud

Os conjuntos de dados contêm transações efetuadas por cartões de crédito em setembro de 2013 por portadores de cartões europeus. Este conjunto de dados apresenta transações que ocorreram em dois dias, onde temos 492 fraudes de 284.807 transações. O conjunto de dados é altamente desequilibrado, a classe positiva (fraudes) representa 0,172% de todas as transações.

Ele contém apenas variáveis ​​numéricas de entrada que são o resultado de uma transformação do PCA. Infelizmente, devido a problemas de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Os recursos V1, V2, ... V28 são os principais componentes obtidos com o PCA, os únicos recursos que não foram transformados com o PCA são 'Tempo' e 'Valor'. Recurso 'Tempo' contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. O recurso 'Valor' é o Montante da transação, esse recurso pode ser usado para o aprendizado de custo-dependente dependente de exemplo. Recurso 'Classe' é a variável de resposta e leva valor 1 no caso de fraude e 0 caso contrário.

Para acompanhar, você precisará de uma instalação do Python com os seguintes pacotes:

NumPy
Pandas
SciKit-Learn

Para começar, vamos começar com as importações necessárias.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.metrics import f1_score, recall_score

Precisamos do NumPy para algumas funções matemáticas básicas e Pandas para ler no arquivo CSV e criar o quadro de dados. Vamos usar um número de sklearn.metrics para avaliar os resultados de nossos modelos.

Em seguida, precisamos criar algumas funções auxiliares. PrintStats irá compilar e exibir os resultados de um modelo. Aqui está o código:

In [3]:
def PrintStats(cmat, y_test, pred):
   # separate out the confusion matrix components
   tpos = cmat[0][0]
   fneg = cmat[1][1]
   fpos = cmat[0][1]
   tneg = cmat[1][0]
   # calculate F!, Recall scores
   f1Score = round(f1_score(y_test, pred), 2)
   recallScore = round(recall_score(y_test, pred), 2)
   # calculate and display metrics
   print(cmat)
   print( 'Accuracy: '+ str(np.round(100*float(tpos+fneg)/float(tpos+fneg + fpos + tneg),2))+'%')
   print( 'Cohen Kappa: '+ str(np.round(cohen_kappa_score(y_test, pred),3)))
   print("Sensitivity/Recall for Model : {recall_score}".format(recall_score = recallScore))
   print("F1 Score for Model : {f1_score}".format(f1_score = f1Score))

<p>O <strong>PrintStats</strong> toma como par&acirc;metros uma matriz de confus&atilde;o, r&oacute;tulos de teste e r&oacute;tulos de previs&atilde;o e faz o seguinte:</p>
<ol>
<li>Separa a matriz de confus&atilde;o em suas partes constituintes.</li>
<li>Calcula as pontua&ccedil;&otilde;es F1, Recall, Accuracy e Cohen Kappa.</li>
<li>Imprime a matriz de confus&atilde;o e todas as pontua&ccedil;&otilde;es calculadas.</li>
</ol>
<p>Tamb&eacute;m precisamos de uma fun&ccedil;&atilde;o, chamada <strong>RunModel</strong>, para realmente treinar o modelo e gerar previs&otilde;es em rela&ccedil;&atilde;o aos dados de teste. Aqui est&aacute; o c&oacute;digo:</p>

In [5]:
def RunModel(model, X_train, y_train, X_test, y_test):
   model.fit(X_train, y_train.values.ravel())
   pred = model.predict(X_test)
   matrix = confusion_matrix(y_test, pred)
   return matrix, pred

A função RunModel toma como entrada o modelo não treinado junto com todos os dados de teste e treinamento, incluindo rótulos. Ele treina o modelo, executa a previsão usando os dados de teste e retorna a matriz de confusão juntamente com os rótulos previstos.

Com essas duas funções criadas, é hora de ver se podemos criar um modelo para fazer a detecção de fraudes. A detecção de fraudes é geralmente considerada um problema de duas classes. Em outras palavras, uma transação é:

Aula 1: não fraude

Ou

Aula 2: fraude

Nosso objetivo é tentar determinar a qual classe uma transação específica pertence. O passo 1 é carregar os dados CSV e criar as classes. Este código irá fazer o truque:

In [8]:
df = pd.read_csv('creditcard.csv')
df.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0


In [9]:
class_names = {0:'Not Fraud', 1:'Fraud'}
print(df.Class.value_counts().rename(index = class_names))

Not Fraud    284315
Fraud           492
Name: Class, dtype: int64


Este é um conjunto de dados bastante típico. De quase 300.000 transações, 492 foram rotuladas como fraudulentas. Pode não parecer muito, mas cada transação representa uma despesa significativa. Juntas, todas essas transações fraudulentas podem representar bilhões de dólares de receita perdida a cada ano. Também representa um problema com a detecção. Uma porcentagem tão pequena das transações fraudulentas torna mais difícil eliminar os infratores do número esmagador de boas transações.

O passo 2 é definir os recursos que queremos usar. Normalmente, queremos aplicar alguma redução de dimensão e engenharia de recursos aos nossos dados, mas isso é outro artigo (ou dois). Em vez disso, usaremos todo o conjunto de dados aqui com o seguinte código:

In [10]:
feature_names = df.iloc[:, 1:30].columns
target = df.iloc[:1, 30: ].columns

data_features = df[feature_names]
data_target = df[target]

Com o conjunto de dados definido, a etapa 3 é dividir os dados em conjuntos de treinamento e teste. Para fazer isso, precisamos importar outra função e executar o seguinte código:

In [11]:
from sklearn.model_selection import train_test_split
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, train_size=0.70, test_size=0.30, random_state=1)

A função train_test_split usa um randomizador para separar os dados em conjuntos de treinamento e teste. 70% dos dados são para treinamento e 30% para testes. A semente aleatória é inicialmente definida para garantir que os mesmos dados sejam usados em todas as execuções.

Para o passo 4, escolhemos uma técnica de aprendizado de máquina ou modelo. Talvez a técnica de aprendizado de máquina de duas classes mais comum seja a regressão logística. Vamos usar isso para este primeiro teste:

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cmat, pred = RunModel(lr, X_train, y_train, X_test, y_test)
PrintStats(cmat, y_test, pred)

C:\Users\Everton\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[85293    15]
 [   57    78]]
Accuracy: 99.92%
Cohen Kappa: 0.684
Sensitivity/Recall for Model : 0.58
F1 Score for Model : 0.68


<p>Voc&ecirc; pode pensar inicialmente que o modelo fez um bom trabalho. Afinal, obteve 99,92% de suas previs&otilde;es corretas. Isso &eacute; verdade, exceto se voc&ecirc; olhar de perto para a matriz de confus&atilde;o, voc&ecirc; ver&aacute; os seguintes resultados:</p>
<p>85293 transa&ccedil;&otilde;es foram classificadas como v&aacute;lidas que eram realmente v&aacute;lidas<br />15 transa&ccedil;&otilde;es foram classificadas como fraude que eram realmente v&aacute;lidas (erro tipo 1)<br />57 transa&ccedil;&otilde;es foram classificadas como v&aacute;lidas e eram fraude (erro tipo 2)<br />78 transa&ccedil;&otilde;es foram classificadas como fraude que eram fraude</p>
<p>Assim, embora a precis&atilde;o fosse &oacute;tima, descobrimos que o algoritmo classificou erroneamente mais de 4 em 10 transa&ccedil;&otilde;es fraudulentas. De fato, se nosso algoritmo simplesmente classificasse tudo como v&aacute;lido, ele teria uma precis&atilde;o melhor que 99,9%, mas seria totalmente in&uacute;til! Portanto, a precis&atilde;o n&atilde;o &eacute; a medida confi&aacute;vel da efic&aacute;cia de um modelo. Em vez disso, analisamos outras medidas como a pontua&ccedil;&atilde;o de Cohen Kappa, Recall e F1. Em cada caso, queremos obter uma pontua&ccedil;&atilde;o o mais pr&oacute;xima poss&iacute;vel de 1.</p>
<p>Talvez outro modelo funcione. Que tal um classificador RandomForest? O c&oacute;digo &eacute; semelhante &agrave; regress&atilde;o log&iacute;stica:</p>

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, n_jobs =4)
cmat, pred = RunModel(rf, X_train, y_train, X_test, y_test)
PrintStats(cmat, y_test, pred)

[[85297    11]
 [   31   104]]
Accuracy: 99.95%
Cohen Kappa: 0.832
Sensitivity/Recall for Model : 0.77
F1 Score for Model : 0.83


Isso é um pouco melhor. Observe que a precisão aumentou ligeiramente, mas as outras pontuações também mostraram melhorias significativas. Assim, uma maneira de melhorar nossa detecção é experimentar modelos diferentes e ver como eles funcionam. Modelos claramente mutáveis ​​ajudaram. Mas existem outras opções também. Um deles está super-amostrando a amostra de registros de fraude ou, inversamente, sub-amostrando a amostra de bons registros. Excesso de amostragem significa adicionar registros de fraude à nossa amostra de treinamento, aumentando assim a proporção geral de registros de fraude. Por outro lado, a subamostragem está removendo registros válidos da amostra, o que tem o mesmo efeito. Alterar a amostragem torna o algoritmo mais "sensível" às transações fraudulentas.

Voltando ao classificador de regressão logística, vamos ver como algumas subamostragens podem melhorar o desempenho geral do modelo. Existem técnicas específicas, como SMOTE e ADASYN, projetadas para amostrar estrategicamente conjuntos de dados desbalanceados. No nosso caso, vamos subexaminar para obter uma divisão uniforme entre fraudes e transações válidas. Isso tornará o conjunto de treinamento muito pequeno, mas o algoritmo não precisa de muitos dados para criar um bom classificador:

In [14]:
fraud_records = len(df[df.Class == 1])
# pull the indicies for fraud and valid rows
fraud_indices = df[df.Class == 1].index
normal_indices = df[df.Class == 0].index
# randomly collect equal samples of each type
under_sample_indices = np.random.choice(normal_indices, fraud_records, False)
df_undersampled = df.iloc[np.concatenate([fraud_indices,under_sample_indices]),:]
X_undersampled = df_undersampled.iloc[:,1:30]
Y_undersampled = df_undersampled.Class
X_undersampled_train, X_undersampled_test, Y_undersampled_train,    Y_undersampled_test = train_test_split(X_undersampled,Y_undersampled,test_size = 0.3)
lr_undersampled = LogisticRegression(C=1)
# run the new model
cmat, pred = RunModel(lr_undersampled, X_undersampled_train, Y_undersampled_train,    X_undersampled_test, Y_undersampled_test)
PrintStats(cmat, Y_undersampled_test, pred)

[[143   9]
 [ 13 131]]
Accuracy: 92.57%
Cohen Kappa: 0.851
Sensitivity/Recall for Model : 0.91
F1 Score for Model : 0.92


C:\Users\Everton\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


A precisão diminuiu, mas todas as outras pontuações aumentaram. Olhando para a matriz de confusão, você pode ver uma porcentagem muito maior de classificações corretas de dados fraudulentos.

Infelizmente, não há almoço grátis. Um número maior de classificações de fraude quase sempre significa um número correspondentemente maior de transações válidas, também classificadas como fraudulentas. Agora, tente o "novo" classificador de regressão logística em relação aos dados de teste originais:

In [15]:
cmat, pred = RunModel(lr_undersampled, X_undersampled_train, Y_undersampled_train, X_test, y_test)
PrintStats(cmat, y_test, pred)

C:\Users\Everton\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[81749  3559]
 [    9   126]]
Accuracy: 95.82%
Cohen Kappa: 0.063
Sensitivity/Recall for Model : 0.93
F1 Score for Model : 0.07


O algoritmo foi muito melhor na captura de transações fraudulentas (16 erros de classificação para 57), mas muito pior na rotulagem incorreta de transações válidas (1551 a 15).

Como cientista de dados, você precisa determinar em que ponto a troca vale a pena. Geralmente, os custos de perder uma transação fraudulenta são muitas vezes maiores do que classificar erroneamente uma boa transação como fraude. Seu trabalho é encontrar o ponto de equilíbrio no treinamento do seu modelo e proceder de acordo.